# How to get Environmental, Social, and Governance data from Refinitiv Data Platform APIs with jupyter/r-notebook Docker image.

The [jupyter/r-notebook](https://jupyter-docker-stacks.readthedocs.io/en/latest/using/selecting.html#jupyter-r-notebook) image provides ready-to-use Jupyter Notebook/JupyterLab application with the [R programming language](https://www.r-project.org/), [R kernel](https://irkernel.github.io/) and popular packages from R ecosystem. Developers can use the jupyter/r-notebook image to create data science/financial works with pre-install packages. 

Developers can start the Jupyter server and access the notebook with the following Docker command:

```
docker run -p 8888:8888 --name notebook -v <project /python/notebook/ directory>:/home/jovyan/work -e JUPYTER_ENABLE_LAB=yes --env-file .env -it jupyter/r-notebook:70178b8e48d7
```
The above command set the following container's options:
- ```-p 8888:8888```: Exposes the server on host port 8888
- ```-v <your working directory>:/home/jovyan/work```: Mounts the working directory on the host as /home/jovyan/work folder in the container to save the files between your host machine and a container.
- ```-e JUPYTER_ENABLE_LAB=yes```: Run JupyterLab instead of the default classic Jupyter Notebook.
- ```--name notebook```: Define a container name as *notebook*
- ```-it```: enable interactive mode with a pseudo-TTY when running a container
- ```--env-file .env```: Pass a ```.env``` file to a container.

*Note*:
- Docker destroys the container and its data when you remove the container, so you always need the ```-v``` option.
- The default notebook username of a container is always **jovyan** (but you can change it to something else)

## Refinitiv Data Platform ESG R Example

Refinitiv Data Platform (RDP) is our cloud-enabled, open platform, that brings together content, analytics, and proprietary, customer, and third-party data distribution and management technology. It provides simple web-based API access to a broad range of content. 

For more detail regarding Refinitiv Data Platform, please see the following APIs resources: 
- [Quick Start](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/quick-start) page.
- [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials) page.
- [RDP APIs: Introduction to the Request-Response API](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#introduction-to-the-request-response-api) page.
- [RDP APIs: Authorization - All about tokens](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#authorization-all-about-tokens) page.

This example demonstrates how to retrieve the ESG (Environmental, Social, and Governance) data from RDP with R on Jupyter Notebook. It uses the [httr](https://cran.r-project.org/web/packages/httr/vignettes/quickstart.html) package to send HTTP request messages.

## The first step is loading the required libraries

- [httr](https://cran.r-project.org/web/packages/httr/index.html): Tools for working with URLs and HTTP.

In [6]:
library(httr)

## Next, set URLs for the Refinitiv Data Platform services

This example uses the following Refinitiv Data Platform services:

- **/auth/oauth2/v1/token**: This service allows clients to get a token for password grant
- **/data/historical-pricing/v1**: This service is used to retrieve Intraday and Interday time-series historical pricing data

This notebook application uses the ```Sys.getenv``` statement to get RDP APIs Auth service endpoint and user's RDP credentials configurations from the environment variables which you have passed to a container with the ```--env-file .env``` option.

In [7]:
REFINITIV_API_URL <- Sys.getenv("RDP_BASE_URL")
OAUTH2_TOKEN_URL <- paste(REFINITIV_API_URL, Sys.getenv("RDP_AUTH_URL"), sep="")

## Then, set the Refinitiv Data Platform credential

The Refinitiv Data Platform credential include:

- **USERNAME**: The resource owner username (typically email)
- **PASSWORD**: The resource owner password
- **CLIENTID**: (Aka Application ID, aka AppKey.) A unique ID defined for an application making the request. Users can generate/manage the AppKey by follow the steps in [Eikon Data API Quick Start](https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/quick-start) page.

In [8]:
USERNAME <- Sys.getenv("RDP_USER")
PASSWORD <- Sys.getenv("RDP_PASSWORD")
CLIENTID <- Sys.getenv("RDP_APP_KEY")

## 1. Login

Login is the first step for all EDP API requests. 

### Construct the body of the HTTP request message

To login, the HTTP post message must be sent to the **auth/oauth2/v1/token** service. The body of the HTTP request message contains the following parameters.

- **grant_type**: Supported values "password" and "refresh_token"
- **username**: The resource owner username (typically email)
- **password**: The resource owner password
- **scope**: The scope of the access request
- **takeExclusiveSignOnControl**: OPTIONAL. This is a Boolean that will allow the API Caller to create a session if the number of concurrent sessions has been reached
- **client_id**: A unique ID defined for an application making the request. Users can generate/manage their application ID's [here](https://emea1.apps.cp.thomsonreuters.com/apps/AppkeyGenerator)

For other parameters, please refer to the [API Documents](http://api.refinitiv.com/).

The following code creates a list that contains those parameter names and values.

In [9]:
RequestTokenBody <- list(grant_type="password",
          username=USERNAME,
          password=PASSWORD,
          scope="trapi",
          takeExclusiveSignOnControl= "True",
          client_id=CLIENTID
          )

### Send the HTTP post message to retrieve tokens

The following code calls the **httr::post** method to send the HTTP post message which contains the login information in the message's body to the **auth/oauth2/v1/token** service. The login information is encoded with a URL encoded form (application/x-www-form-urlencoded) in the message's body. The HTTP post message also contains the **Authorization** HTTP header with the client id as its value. 

After receiving the response, it calls the **stop_for_status** function which converts HTTP errors to R errors or warnings. If the request was successful, it prints the response on the screen. Otherwise, it will stop.

In [10]:
RequestTokenResponse <- httr::POST(OAUTH2_TOKEN_URL,
                 add_headers(Authorization = CLIENTID),                
                 body = RequestTokenBody,
                 encode = "form")
stop_for_status(RequestTokenResponse)
RequestTokenResponse

Response [https://api.refinitiv.com/auth/oauth2/v1/token]
  Date: 2021-09-10 05:22
  Status: 200
  Content-Type: application/json
  Size: 8.13 kB


If the response status is not 200, please verify the URL or login information.

### Get an access token

If the response status is 200, the HTTP response will contain an access token. This access token will be used in subsequent requests. 

The data is in JSON format so it calls the **httr::content** method to extract the JSON content from the HTTP response. Then, it saves an access token into the **ACCESS_TOKEN** variable so it can be used in the subsequent requests.

In [11]:
TokenContent <- httr::content(RequestTokenResponse, "parsed", "application/json", encoding="UTF-8")
ACCESS_TOKEN <- TokenContent$access_token
ACCESS_TOKEN

[1] "eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImJlcGpHV0dkOW44WU9VQ1NwX3M3SXlRMmlKMFkzeWRFaHo1VDJJVlNqWTgifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwicWttNmlkZFN2V2VRV3Y3cnlWZ1lDWTVSZTBwb2JPa2h6bWN6Rk1lb0NaOGJfNm1Qb2FYdnJmWm5PUmd1RkNQZnNwbzkyVXhxT1c0V1A2SHFPVGJBblItRV9mNlc0ZzJZM2YzVUVRZjcxeTZoOEoxNUI0Y01YWUQ1Rng0WlIzN184ZXpoTnFFR1BSTTMwOWFKOUI2X3QtdGlJbEZUMlBXQjJicHgtYjR0N2xtT1c1UnlBTlhGYlphRDNXckI2WmNuUXplSFFTZXU0eDR1aHRfNFA4LVJzYWZMQTlRWUQxMXhCSFA2akdfMFc3bVNTZnh6blFGUTVPRVNHV0diLW5PTUZnU0lDUWVTYVMxazF2Q1VRTVN0NjctNTFXR0FWSEgzRFg5X015RS03azJhX2drQnQxS0hEbnF1b0VKeTVUS2QydmhOaThMa0JHQkhweXBoU1pkOHFCQ3g2MnNHVHBaS1ZhYVppZ3NQaG1NSUFMM2plUFE5ekZJc0RZRmJsRTAxTmxLcE51VURFS1ozVlV3YWNNNVp2cFlSX0xfbml3WjQyYUF0cUFpQW9SS0ZxWUdjSXRGQkNSSEt2RkkwVkg0aGFhX2JockdUUWxsdm9RNGlyZURvVjhlT19uQzcyaThia3FnOEh3bjJkeFF1U1BMMERzYmZUcmRXcUhRalBycEtIanRPaWRkRVFNVDdjTzQ4SmtvZXpGbGtpS3FCNEtZdnh2MWxFdTN1UW1aM0ZlVUhZZzVYT25NdjdqbzY0akEzZzBsOFhBN19DYTNYNzA2T01mR0s2N2ZNbW95eFowQVNDbGRoV1Y5cUd6bDQyMWlZMExOMmp0cXhxSFM0YkFNTF9INElTSXE4WVlzRUdiQldkY0dXSG1yM01uZl9wQmFjc1BzRWRqMEd3alV5bW1VQU4ySkFYMUgxTmpHNlA0MTV5MUtBd0NPdjhuZzRrUVdpOU9MOVl2OWMtUXRBRlRpazI1MklHbDJvV1JVWWxfY01Ob3l0UmFlaU5MVV9LckdSVHJKOUxqNy1iaXV5TGgzOWJpemxNY2Zibnpqcndhb0JBYmNWSksyNmVQZzFTLVE1b3pUbkJWLUMyMlBKaTdsYTVzdTJNUVh3OFQteW1SR05nWkxUS3R2LTUtbmJHeXI4ZFoxOEJiNjhXRFE4bklBTnJfQWYtN1hsd1AxcFQ5X1FRd2R6TW40YW5RUUtoSFl2YWZ2aWs2aHk5cHo2eFY0NTZqMGQzN2xxVEJqY2dmeXZxbGhGSG1DMkJqQldUeXZtakxxUzBvazRzcWNfREdWTW8teVFLaGp6Y2Y0azRDZnNjVmtaRVdVdTRlWnFXVUxrQjY1MFYwdnItNlFnaHIwYVRpeE9sMzJDSTFZQmVsVGFhalc4STg5WV9fbHV2NFRIRG14bDRvam85QUgxZU1YSTV4ZTlsaUg4VXdXSjVDRWZRSXlISDFxYTFMT0dUMWQ5c0Z2czBpYml0dUhYZFFKekJZaVJ5Wm1pdk9tN2VhTVFmZFY4TjlkVjZzUUhPWHhKcHZuNi1ESkp5WXVDblJ4LWE3cmtic0hNMjV1SUFjNXhzWTlyQmkxQ241OEppUFFmS2F0MUNhckV6TER3ZzhXcXVtOERzVHNMNUtFRVNCYnhWMEhzcVBGa2c4NmZuclBXV3NsWTNqOW1KVTFEc29RZHFodGVkMTItMzdJN0JSZ0xFRUtNSHlsSlpURWVabHZaMFZoa2UyU3J2R1lwb185ODg5dUhhT21UNEw4MjRkNHBhUGV5VWU0b2tMbU5LMVR3RzNoUGltOWdCU3h6aV9HdUpOZ2JMMW5lNi1RWk10SVB2dGNXLWVfV0pkVFlid1BmTHpUYjJEVEZHT2EzQnA4WXFqTlhWVTMxVlBadmU1dGZnR3VoRDJXdjVoeEZudWdEc2FSSVV6MnZ5aFRHLTJxanF1MXF5R252YlZ2cWdWMlgybFpuZHp4VmN1Y2dXbDhrdjNWWXBUcDQxMzR2UEg2aFh4T0ZxcjFHbWRXczhuaG15YzNQMVVtREJQeG5tc2luV2NrUDJod0phc05iWVM3eUZXUUZYNG1BMkFDN3lIZGtZSnoyXCIsXCJpdlwiOlwiT1Z5aUFSdlhORFQ5eXRnUlwiLFwicHJvdGVjdGVkXCI6XCJleUpoYkdjaU9pSkJWMU5mUlU1RFgxTkVTMTlCTWpVMklpd2laVzVqSWpvaVFUSTFOa2REVFNJc0lucHBjQ0k2SWtSRlJpSjlcIixcInJlY2lwaWVudHNcIjpbe1wiZW5jcnlwdGVkX2tleVwiOlwiQVFJQkFIaExHNjU4XzJxZ1JMRGVwbHQtS1Y5MVAtMlFSSHJjS1MxdnI3cjNXclpCOWdGME9xNkNOaldVSkljbmhaeTlJT3ZfQUFBQWZqQjhCZ2txaGtpRzl3MEJCd2FnYnpCdEFnRUFNR2dHQ1NxR1NJYjNEUUVIQVRBZUJnbGdoa2dCWlFNRUFTNHdFUVFNUnl1aGE5YVg5SDlkOGdOc0FnRVFnRHZQUUJubnZTSXR5UklseGd6SFF5TVVyalYxbk1TZWgtYjYzT2NIQTk3djh3dlZ2cTlLRU5yT0ZTSThkelE2dmFfRDFDQnRMSVlQX0V0Z0RBXCIsXCJoZWFkZXJcIjp7XCJraWRcIjpcImFybjphd3M6a21zOmFwLXNvdXRoZWFzdC0xOjg5ODA4NDk4NDg3NzprZXkvZTQ4NzBiMWMtOWNjNy00NDc4LWI5NTAtYzU2NTliNWM1NjhmXCJ9fSx7XCJlbmNyeXB0ZWRfa2V5XCI6XCJBUUlDQUhoV3ZjSFhyTVlXVXlkVnJsZzV1TWxfUWlINmdLdHNabmRYQlgyakhuNjkzd0ZBN3NZOXhUUFhRQ1UyeVIzV0gyT2tBQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU03ZUhaOHZhWkNJMmx1a1lmQWdFUWdEdlY1RXIwdXR1TGxqU055QWlUVzMxckpKVVZNeUJPZkF0VGp4bzN6YmZLQkNfWVJSN3Z0TlduOTUta0VUTTlMV2NWd3FPS2xYUW15eUN2R2dcIixcImhlYWRlclwiOntcImtpZFwiOlwiYXJuOmF3czprbXM6ZXUtd2VzdC0xOjg5ODA4NDk4NDg3NzprZXkvNmE4NjU5MmYtMzYxNC00ODE2LTljNDMtODExYmVmZGE2ZDYwXCJ9fSx7XCJlbmNyeXB0ZWRfa2V5XCI6XCJBUUlDQUhpT1JjYVBoX2laM21rakVtUTZBaHpDOWJHeHJJNF9XaFZIWDVPRnI2NTgwUUU0UnlwSFl3bnl2RlpCQkdLQnRUM1JBQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU1TcnltTEJoQnNHV1VsODVrQWdFUWdEc1ktRmszYl96eUhDa1JTR2JabUVxcmc5TE9ZNWowbG84R1p1dU5jVHNIdHJzNlVCcTFBcDhPNGF0Q0s1akZmZEIzY2d2bnRPNUliazg3RWdcIixcImhlYWRlclwiOntcImtpZFwiOlwiYXJuOmF3czprbXM6dXMtZWFzdC0xOjg5ODA4NDk4NDg3NzprZXkvMWZmZjY3OTMtNjVkZS00N2M0LWI3NjYtNzZjZDNjMTk0ZWYxXCJ9fV0sXCJ0YWdcIjpcImY1VHZQak85eDdJdDB4cmxndGhQdndcIn0iLCJyczEiOiIwMDVmOGY1ZGQw

After the application received the Access Token (an authorization token) from RDP Auth Service, all subsequent REST API calls will use this token to get the data. The application needs to input Access Token via *Authorization* HTTP request message header as shown below. 
- Header: 
    * Authorization = ```Bearer <RDP Access Token>```

Please notice *the space* between the ```Bearer``` and ```RDP Access Token``` values.

The next step is requesting ESG (Environmental, Social, and Governance) data from RDP. We use the ESG scores-full API endpoint which provides full coverage of Refinitiv's proprietary ESG Scores with full history for consumers as an example API.

We get the RDP ESG Service API endpoint from a ```.env``` file.

In [12]:
ESG_URL <- paste(REFINITIV_API_URL, Sys.getenv("RDP_ESG_URL"), sep="")

In [13]:
ESGResponse <- httr::GET(paste(ESG_URL,sep=""),
                                add_headers(Authorization = paste(TokenContent[["token_type"]], ACCESS_TOKEN)),
                                query=list(universe="TSLA.O")
                 )  
stop_for_status(ESGResponse)
ESGResponse   

Response [https://api.refinitiv.com/data/environmental-social-governance/v2/views/scores-full?universe=TSLA.O]
  Date: 2021-09-10 05:22
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 16.7 kB


Then display data in JSON message format.

In [14]:
ESGContent <- httr::content(ESGResponse, "parsed", "application/json", encoding="UTF-8")
ESGContent

$links
$links$count
[1] 10


$variability
[1] "variable"

$universe
$universe[[1]]
$universe[[1]]$Instrument
[1] "TSLA.O"

$universe[[1]]$`Company Common Name`
[1] "Tesla Inc"

$universe[[1]]$`Organization PermID`
[1] "4297089638"

$universe[[1]]$`Reporting Currency`
[1] "USD"



$data
$data[[1]]
$data[[1]][[1]]
[1] "TSLA.O"

$data[[1]][[2]]
[1] "2019-12-31"

$data[[1]][[3]]
[1] 58.13081

$data[[1]][[4]]
[1] 32.11419

$data[[1]][[5]]
[1] 6.097561

$data[[1]][[6]]
[1] 45.88597

$data[[1]][[7]]
[1] 64.50822

$data[[1]][[8]]
[1] 68.49285

$data[[1]][[9]]
[1] 68.71345

$data[[1]][[10]]
[1] 44.47674

$data[[1]][[11]]
[1] 82.79221

$data[[1]][[12]]
[1] 50.5291

$data[[1]][[13]]
[1] 24.35897

$data[[1]][[14]]
[1] 75.66138

$data[[1]][[15]]
[1] 48.85714

$data[[1]][[16]]
[1] 69.17697

$data[[1]][[17]]
[1] 49.9512

$data[[1]][[18]]
[1] 63

$data[[1]][[19]]
[1] "B-"

$data[[1]][[20]]
[1] "C-"

$data[[1]][[21]]
[1] "D-"

$data[[1]][[22]]
[1] "C+"

$data[[1]][[23]]
[1] "B"

$data[[1]][[24]]
[1] "B+"

$data[[1]][[25]]
[1] "B+"

$data[[1]][[26]]
[1] "C+"

$data[[1]][[27]]
[1] "A-"

$data[[1]][[28]]
[1] "B-"

$data[[1]][[29]]
[1] "D+"

$data[[1]][[30]]
[1] "A-"

$data[[1]][[31]]
[1] "C+"

$data[[1]][[32]]
[1] "B+"

$data[[1]][[33]]
[1] "C+"

$data[[1]][[34]]
[1] "B"

$data[[1]][[35]]
[1] 100

$data[[1]][[36]]
[1] "2021-08-21T00:00:00"


$data[[2]]
$data[[2]][[1]]
[1] "TSLA.O"

$data[[2]][[2]]
[1] "2018-12-31"

$data[[2]][[3]]
[1] 51.11984

$data[[2]][[4]]
[1] 27.83265

$data[[2]][[5]]
[1] 4.545455

$data[[2]][[6]]
[1] 43.25371

$data[[2]][[7]]
[1] 31.41834

$data[[2]][[8]]
[1] 74.77527

$data[[2]][[9]]
[1] 73.7013

$data[[2]][[10]]
[1] 46.68874

$data[[2]][[11]]
[1] 92.16418

$data[[2]][[12]]
[1] 32.83133

$data[[2]][[13]]
[1] 25

$data[[2]][[14]]
[1] 79.51807

$data[[2]][[15]]
[1] 52.27273

$data[[2]][[16]]
[1] 31.37631

$data[[2]][[17]]
[1] 20.12195

$data[[2]][[18]]
[1] 48.57313

$data[[2]][[19]]
[1] "B-"

$data[[2]][[20]]
[1] "C-"

$data[[2]][[21]]
[1] "D-"

$data[[2]][[22]]
[1] "C+"

$data[[2]][[23]]
[1] "C-"

$data[[2]][[24]]
[1] "B+"

$data[[2]][[25]]
[1] "B+"

$data[[2]][[26]]
[1] "C+"

$data[[2]][[27]]
[1] "A+"

$data[[2]][[28]]
[1] "C-"

$data[[2]][[29]]
[1] "D+"

$data[[2]][[30]]
[1] "A-"

$data[[2]][[31]]
[1] "B-"

$data[[2]][[32]]
[1] "C-"

$data[[2]][[33]]
[1] "D+"

$data[[2]][[34]]
[1] "C+"

$data[[2]][[35]]
[1] 100

$data[[2]][[36]]
[1] "2020-11-06T00:00:00"


$data[[3]]
$data[[3]][[1]]
[1] "TSLA.O"

$data[[3]][[2]]
[1] "2017-12-31"

$data[[3]][[3]]
[1] 39.08362

$data[[3]][[4]]
[1] 31.12717

$data[[3]][[5]]
[1] 23.17073

$data[[3]][[6]]
[1] 30.34796

$data[[3]][[7]]
[1] 29.87936

$data[[3]][[8]]
[1] 56.30034

$data[[3]][[9]]
[1] 18.56061

$data[[3]][[10]]
[1] 30.85938

$data[[3]][[11]]
[1] 90.43478

$data[[3]][[12]]
[1] 11.30137

$data[[3]][[13]]
[1] 27.61905

$data[[3]][[14]]
[1] 47.26027

$data[[3]][[15]]
[1] 41.41791

$data[[3]][[16]]
[1] 31.40036

$data[[3]][[17]]
[1] 31.25673

$data[[3]][[18]]
[1] 20.20833

$data[[3]][[19]]
[1] "C"

$data[[3]][[20]]
[1] "C-"

$data[[3]][[21]]
[1] "D+"

$data[[3]][[22]]
[1] "C-"

$data[[3]][[23]]
[1] "C-"

$data[[3]][[24]]
[1] "B-"

$data[[3]][[25]]
[1] "D+"

$data[[3]][[26]]
[1] "C-"

$data[[3]][[27]]
[1] "A"

$data[[3]][[28]]
[1] "D"

$data[[3]][[29]]
[1] "C-"

$data[[3]][[30]]
[1] "C+"

$data[[3]][[31]]
[1] "C"

$data[[3]][[32]]
[1] "C-"

$data[[3]][[33]]
[1] "C-"

$data[[3]][[34]]
[1] "D+"

$data[[3]][[35]]
NULL

$data[[3]][[36]]
[1] "2020-09-18T00:00:00"


$data[[4]]
$data[[4]][[1]]
[1] "TSLA.O"

$data[[4]][[2]]
[1] "2016-12-31"

$data[[4]][[3]]
[1] 32.81124

$data[[4]][[4]]
[1] 28.12437

$data[[4]][[5]]
[1] 23.4375

$data[[4]][[6]]
[1] 23.4809

$data[[4]][[7]]
[1] 39.22855

$data[[4]][[8]]
[1] 39.59417

$data[[4]][[9]]
[1] 12.7193

$data[[4]][[10]]
[1] 25.22936

$data[[4]][[11]]
[1] 61.65049

$data[[4]][[12]]
[1] 27.95276

$data[[4]][[13]]
[1] 0

$data[[4]][[14]]
[1] 47.6378

$data[[4]][[15]]
[1] 35.21739

$data[[4]][[16]]
[1] 44.33719

$data[[4]][[17]]
[1] 35.92879

$data[[4]][[18

## Conclusion

The jupyter/r-notebook image provides Jupyter Server with a handful of libraries that is enough for building a financial, data science notebook application with the R programming language.

## <a id="references"></a>References

You can find more details regarding the Refinitiv Data Platform APIs, Jupyter Docker Stacks, and related technologies for this notebook from the following resources:
* [Refinitiv Data Platform APIs page](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis) on the [Refinitiv Developer Community](https://developers.refinitiv.com/) website.
* [Refinitiv Data Platform APIs Playground page](https://api.refinitiv.com).
* [Refinitiv Data Platform APIs: Introduction to the Request-Response API](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#introduction-to-the-request-response-api).
* [Refinitiv Data Platform APIs: Authorization - All about tokens](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/tutorials#authorization-all-about-tokens).
* [Setup Jupyter Notebook for R](https://developers.refinitiv.com/en/article-catalog/article/setup-jupyter-notebook-r) article.
* [Jupyter Dokcer Stacks page](https://jupyter-docker-stacks.readthedocs.io/en/latest/index.html)
* [Jupyter Dokcer Stack on DockerHub](https://hub.docker.com/u/jupyter) website

For any questions related to RRefinitiv Data Platform APIs, please use the following forum on the [the Developers Community Q&A page](https://community.developers.refinitiv.com/).
- [RDP APIs Forum](https://community.developers.refinitiv.com/spaces/231/index.html).